# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.data.dataset_factory import TabularDatasetFactory
import pandas as pd
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.model import Model
from azureml.core.model import InferenceConfig
from azureml.core import Workspace, Environment
from azureml.core import Model
from azureml.core.webservice import AciWebservice, Webservice
import json
import joblib

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
#Connection to the workspace and definition of the experiment 
ws = Workspace.from_config()
experiment_name = 'Parkinson-classification'

experiment=Experiment(workspace=ws, name=experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

Workspace name: quick-starts-ws-137760
Azure region: southcentralus
Subscription id: 61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30
Resource group: aml-quickstarts-137760


In [3]:
#Get the data from github
path_url = 'https://gist.githubusercontent.com/Nwaneto/0d1477bd10c92f8b16ab19306d21a17f/raw/0af3078c0d174e26039ab31525487ceaceda77b0/parkinson-classification-data.csv'
ds = TabularDatasetFactory.from_delimited_files(path = path_url)
ds.to_pandas_dataframe().head()

,name,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,...,Shimmer:DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE
0,phon_R01_S01_1,119.992,157.302,74.997,0.00784,0.00007,0.00370,0.00554,0.01109,0.04374,...,0.06545,0.02211,21.033,1,0.414783,0.815285,-4.813031,0.266482,2.301442,0.284654
1,phon_R01_S01_2,122.400,148.650,113.819,0.00968,0.00008,0.00465,0.00696,0.01394,0.06134,...,0.09403,0.01929,19.085,1,0.458359,0.819521,-4.075192,0.335590,2.486855,0.368674
2,phon_R01_S01_3,116.682,131.111,111.555,0.01050,0.00009,0.00544,0.00781,0.01633,0.05233,...,0.08270,0.01309,20.651,1,0.429895,0.825288,-4.443179,0.311173,2.342259,0.332634
3,phon_R01_S01_4,116.676,137.871,111.366,0.00997,0.00009,0.00502,0.00698,0.01505,0.05492,...,0.08771,0.01353,20.644,1,0.434969,0.819235,-4.117501,0.334147,2.405554,0.368975
4,phon_R01_S01_5,116.014,141.781,110.655,0.01284,0.00011,0.00655,0.00908,0.01966,0.06425,...,0.10470,0.01767,19.649,1,0.417356,0.823484,-3.747787,0.234513,2.332180,0.410335


In [4]:
df=ds.to_pandas_dataframe()
df.drop("name", inplace=True, axis=1)  
df.to_csv("./training_data.csv")
datastore = ws.get_default_datastore()
datastore.upload(src_dir = "./", target_path = "data/")
training_data = TabularDatasetFactory.from_delimited_files(path = [(datastore, ("data/training_data.csv"))])


Uploading an estimated of 13 files
Uploading ./.amlignore
Uploaded ./.amlignore, 1 files out of an estimated total of 13
Uploading ./.amlignore.amltmp
Uploaded ./.amlignore.amltmp, 2 files out of an estimated total of 13
Uploading ./automl.ipynb
Uploaded ./automl.ipynb, 3 files out of an estimated total of 13
Uploading ./automl2.ipynb
Uploaded ./automl2.ipynb, 4 files out of an estimated total of 13
Uploading ./hyperparameter_tuning2.ipynb
Uploaded ./hyperparameter_tuning2.ipynb, 5 files out of an estimated total of 13
Uploading ./README.md
Uploaded ./README.md, 6 files out of an estimated total of 13
Uploading ./train.py
Uploaded ./train.py, 7 files out of an estimated total of 13
Uploading ./training_data.csv
Uploaded ./training_data.csv, 8 files out of an estimated total of 13
Uploading ./hyperparameter_tuning.ipynb
Uploaded ./hyperparameter_tuning.ipynb, 9 files out of an estimated total of 13
Uploading ./.ipynb_checkpoints/automl-checkpoint.ipynb
Uploaded ./.ipynb_checkpoints/auto

In [5]:
df.head()

,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,MDVP:Shimmer(dB),...,Shimmer:DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE
0,119.992,157.302,74.997,0.00784,0.00007,0.00370,0.00554,0.01109,0.04374,0.426,...,0.06545,0.02211,21.033,1,0.414783,0.815285,-4.813031,0.266482,2.301442,0.284654
1,122.400,148.650,113.819,0.00968,0.00008,0.00465,0.00696,0.01394,0.06134,0.626,...,0.09403,0.01929,19.085,1,0.458359,0.819521,-4.075192,0.335590,2.486855,0.368674
2,116.682,131.111,111.555,0.01050,0.00009,0.00544,0.00781,0.01633,0.05233,0.482,...,0.08270,0.01309,20.651,1,0.429895,0.825288,-4.443179,0.311173,2.342259,0.332634
3,116.676,137.871,111.366,0.00997,0.00009,0.00502,0.00698,0.01505,0.05492,0.517,...,0.08771,0.01353,20.644,1,0.434969,0.819235,-4.117501,0.334147,2.405554,0.368975
4,116.014,141.781,110.655,0.01284,0.00011,0.00655,0.00908,0.01966,0.06425,0.584,...,0.10470,0.01767,19.649,1,0.417356,0.823484,-3.747787,0.234513,2.332180,0.410335


In [6]:
#Creation of the compute-cluster
amlcompute_cluster_name = "cpu-cluster"

try:
    aml_compute = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS3_V2',
                                                           max_nodes=5)
    aml_compute = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

aml_compute.wait_for_completion(show_output=True , min_node_count = 1, timeout_in_minutes = 2)

Creating
Succeeded.....................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [7]:
# Automl settings 
automl_settings = {
    "experiment_timeout_minutes" :15,
    "max_concurrent_iterations": 4,
    "n_cross_validations": 5,
    "primary_metric": 'accuracy',
}

# Automl config 
automl_config = AutoMLConfig(
    task="classification",
    compute_target=aml_compute,
    training_data=training_data,
    label_column_name="status",
    **automl_settings
)

In [8]:
# Submit experiment
auto_ml_run = experiment.submit(config = automl_config, show_output = True)


Running on remote.
No run_configuration provided, running on cpu-cluster with default configuration
Running on remote compute: cpu-cluster
Parent Run ID: AutoML_a647e683-2a75-47a6-8915-7892ca77fef2

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: 

        46   SparseNormalizer XGBoostClassifier             0:00:27          nan    0.9692
ERROR: {
    "additional_properties": {},
    "error": {
        "additional_properties": {
            "debugInfo": null
        },
        "code": "UserError",
        "severity": null,
        "message": "Experiment timeout reached, please consider increasing your experiment timeout.",
        "message_format": "Experiment timeout reached, please consider increasing your experiment timeout.",
        "message_parameters": {},
        "reference_code": null,
        "details_uri": null,
        "target": null,
        "details": [],
        "inner_error": {
            "additional_properties": {},
            "code": "ResourceExhausted",
            "inner_error": {
                "additional_properties": {},
                "code": "Timeout",
                "inner_error": {
                    "additional_properties": {},
                    "code": "ExperimentTimeoutForIterations",
        

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [9]:
RunDetails(auto_ml_run).show()
auto_ml_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more abo

ERROR: {
    "additional_properties": {},
    "error": {
        "additional_properties": {
            "debugInfo": null
        },
        "code": "UserError",
        "severity": null,
        "message": "Experiment timeout reached, please consider increasing your experiment timeout.",
        "message_format": "Experiment timeout reached, please consider increasing your experiment timeout.",
        "message_parameters": {},
        "reference_code": null,
        "details_uri": null,
        "target": null,
        "details": [],
        "inner_error": {
            "additional_properties": {},
            "code": "ResourceExhausted",
            "inner_error": {
                "additional_properties": {},
                "code": "Timeout",
                "inner_error": {
                    "additional_properties": {},
                    "code": "ExperimentTimeoutForIterations",
                    "inner_error": null
                }
            }
        }
    },
    "corre

{'runId': 'AutoML_a647e683-2a75-47a6-8915-7892ca77fef2',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-02-07T07:39:40.13579Z',
 'endTimeUtc': '2021-02-07T08:04:20.696772Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'cpu-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"Parkinson-classification","subscription_id":"61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30","resource_group":"aml-quickstarts-137760","workspace_name":"quick-starts-ws-137760","region":"southcentralus","compute_target":"cpu-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":null,"y_max":null,"num_classe

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [10]:
# Retrieve the best automl model.
best_run, fitted_model = auto_ml_run.get_output()
print(best_run)
print(fitted_model)

#Register the model
description ='Parkinson detection classification '
model_name='Parkinson-detection-automl'
model_path='./'
tags = None
model = auto_ml_run.register_model(model_name = model_name, description = description , tags = tags)
print(auto_ml_run.model_id)

Package:azureml-automl-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-core, training version:1.21.0.post1, current version:1.20.0
Package:azureml-dataprep, training version:2.8.2, current version:2.7.3
Package:azureml-dataprep-native, training version:28.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.6.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-defaults, training version:1.21.0, current version:1.20.0
Package:azureml-interpret, training version:1.21.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.21.0, current version:1.20.0
Package:azureml-telemetry, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.21.0, current version:1.20.0


Run(Experiment: Parkinson-classification,
Id: AutoML_a647e683-2a75-47a6-8915-7892ca77fef2_47,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                objective=None,
                                                                                                random_state=None,
                           

In [11]:
#TODO: Save the best modelimport os
os.makedirs('./amlmodel', exist_ok=True)

best_run.download_file('/outputs/model.pkl',os.path.join('./amlmodel','automl_best_model_park.pkl'))

for f in best_run.get_file_names():
    if f.startswith('outputs'):
        output_file_path = os.path.join('./amlmodel', f.split('/')[-1])
        print(f'Downloading from {f} to {output_file_path} ...')
        best_run.download_file(name=f, output_file_path=output_file_path)

In [12]:
#Register the best model for the deployement

model=best_run.register_model(
            model_name = 'automl-bestmodel-parkinson', 
            model_path = './outputs/model.pkl',
            model_framework=Model.Framework.SCIKITLEARN,
            description='Parkinson detection'
)

In [13]:
# Download the conda environment file and define the environement
best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'conda_env.yml')
myenv = Environment.from_conda_specification(name = 'myenv',
                                             file_path = 'conda_env.yml')

In [14]:
# download the scoring file produced by AutoML
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'score_auto_park.py')

# set inference config
inference_config = InferenceConfig(entry_script= 'score_auto_park.py',
                                    environment=myenv)

In [15]:
# set Aci Webservice config
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, auth_enabled=True)

In [16]:
# deploye the model as a web service 
service = Model.deploy(workspace=ws, 
                       name='automl-bestmodel-parkinso', 
                       models=[model], 
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)

In [17]:
service

AciWebservice(workspace=Workspace.create(name='quick-starts-ws-137760', subscription_id='61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30', resource_group='aml-quickstarts-137760'), name=automl-bestmodel-parkinso, image_id=None, compute_type=None, state=ACI, scoring_uri=Transitioning, tags=None, properties={}, created_by={})

In [18]:
# wait for deployment to finish and display the scoring uri and swagger uri
service.wait_for_deployment(show_output=True)

print('Service state:')
print(service.state)

print('Scoring URI:')
print(service.scoring_uri)

print('Swagger URI:')
print(service.swagger_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running......................................................................................................................................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Service state:
Healthy
Scoring URI:
http://26ea3b2a-396e-4404-baa4-ec904de156b9.southcentralus.azurecontainer.io/score
Swagger URI:
http://26ea3b2a-396e-4404-baa4-ec904de156b9.southcentralus.azurecontainer.io/swagger.json


In [19]:
x_df=df.sample(3)
y_df = x_df.pop('status')

x_df['Column1'] = 0.0

# convert the records to a json data file
recored=x_df.to_dict(orient='records')

scoring_json = json.dumps({'data': recored})
print(scoring_json)

{"data": [{"MDVP:Fo(Hz)": 112.547, "MDVP:Fhi(Hz)": 133.374, "MDVP:Flo(Hz)": 105.715, "MDVP:Jitter(%)": 0.00355, "MDVP:Jitter(Abs)": 3e-05, "MDVP:RAP": 0.00166, "MDVP:PPQ": 0.0019, "Jitter:DDP": 0.00499, "MDVP:Shimmer": 0.01358, "MDVP:Shimmer(dB)": 0.129, "Shimmer:APQ3": 0.00664, "Shimmer:APQ5": 0.00786, "MDVP:APQ": 0.0114, "Shimmer:DDA": 0.01992, "NHR": 0.00435, "HNR": 26.436, "RPDE": 0.413295, "DFA": 0.7567, "spread1": -6.323531, "spread2": 0.218885, "D2": 2.193412, "PPE": 0.160376, "Column1": 0.0}, {"MDVP:Fo(Hz)": 252.455, "MDVP:Fhi(Hz)": 261.487, "MDVP:Flo(Hz)": 182.786, "MDVP:Jitter(%)": 0.00185, "MDVP:Jitter(Abs)": 7e-06, "MDVP:RAP": 0.00092, "MDVP:PPQ": 0.00113, "Jitter:DDP": 0.00276, "MDVP:Shimmer": 0.01152, "MDVP:Shimmer(dB)": 0.103, "Shimmer:APQ3": 0.00614, "Shimmer:APQ5": 0.0073, "MDVP:APQ": 0.0086, "Shimmer:DDA": 0.01841, "NHR": 0.00432, "HNR": 26.805, "RPDE": 0.610367, "DFA": 0.635204, "spread1": -7.31951, "spread2": 0.200873, "D2": 2.028612, "PPE": 0.086398, "Column1": 0.0

In [20]:
output = service.run(scoring_json)

In [21]:
print (output)

{"result": [1, 0, 1]}


In [22]:
print(y_df)

171    0
47     0
158    1
Name: status, dtype: int64


In [23]:
print(service.get_logs())

2021-02-07T08:25:39,787006637+00:00 - iot-server/run 
2021-02-07T08:25:39,786983937+00:00 - rsyslog/run 
2021-02-07T08:25:39,787200839+00:00 - gunicorn/run 
2021-02-07T08:25:39,796514520+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_7785023fceb74e4facc1b1a577b1faf9/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7785023fceb74e4facc1b1a577b1faf9/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7785023fceb74e4facc1b1a577b1faf9/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7785023fceb74e4facc1b1a577b1faf9/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7785023fceb74e4facc1b1a577b1faf9/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd

In [ ]:
service.delete()
aml_compute.delete()

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service